In [ ]:
!git clone https://github.com/quangster/machine-translation

Cloning into 'machine-translation'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 1), reused 19 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 5.98 MiB | 5.75 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/machine-translation


In [ ]:
%cd machine-translation

In [ ]:
!pip install -q gdown
!gdown --folder 1cPdLNnTlsj3N1FE9x6_K608bCAaYaVGM -O data

In [ ]:
!pip install -q underthesea

In [1]:
from src.data import Vocabulary, EnTokenizer, ViTokenizer, MTDataset
import pickle
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/quangster/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Lưu ý khi tokenize: vì các bộ vocab được build bằng chữ thường nên lúc tokenize cũng cần chuyển về chữ thường bằng hàm .lower()

In [2]:
# vietnamese tokenizer
vi_tokenizer = ViTokenizer()
vi_tokenizer.tokenize("   Ðảm baỏ chất lựơng phòng  , thí nghịêm       hoá học".lower())

['đảm', 'bảo', 'chất', 'lượng', 'phòng', ',', 'thí', 'nghiệm', 'hóa', 'học']

In [3]:
# english tokenizer
en_tokenizer = EnTokenizer()
en_tokenizer.tokenize("Hello,     world!, I'm a student.".lower())

['hello', ',', 'world', '!', ',', 'i', "'m", 'a', 'student', '.']

In [4]:
en_vocab = Vocabulary.load('./ckpts/en_vocab.json')
vi_vocab = Vocabulary.load('./ckpts/vi_vocab.json')
print(en_vocab)
print(vi_vocab)

Vocabulary[language=english, size=34687]
Vocabulary[language=vietnamese, size=21681]


quá trình build dataset có lược bớt các câu dài hơn max_length, và loại bỏ các câu rỗng nên sẽ có số lượng ít hơn so với inputs nhận được

In [5]:
def get_dataset():
    global en_vocab, vi_vocab

    with open("./data/train.pkl", "rb") as f:
        train = pickle.load(f)
        train_en_ids, train_vi_ids = train[0], train[1]

    with open("./data/dev.pkl", "rb") as f:
        dev = pickle.load(f)
        dev_en_ids, dev_vi_ids = dev[0], dev[1]

    ####
    # config length of training dataset or length of sequence here 
    #### 

    train_dataset = MTDataset(
        train_en_ids[0:200000], 
        train_vi_ids[0:200000],
        max_length=20,
        padding_idx=en_vocab['<pad>'],
    )
    
    val_dataset = MTDataset(
        dev_en_ids, 
        dev_vi_ids,
        max_length=20,
        padding_idx=en_vocab['<pad>'],
    )

    return train_dataset, val_dataset

In [6]:
train_dataset, val_dataset = get_dataset()
len(train_dataset), len(val_dataset)

(94104, 11668)

In [7]:
# double check
X, y = val_dataset[56]

print(en_vocab.indexes2words(X.numpy()))
print(vi_vocab.indexes2words(y.numpy()))

['<sos>', 'and', 'in', 'those', 'four', 'gestures', 'is', 'the', 'cycle', 'of', 'life', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'và', 'trong', '4', 'cử', 'chỉ', 'tay', 'này', 'vòng', 'xoay', 'của', 'cuộc', 'sống', 'được', 'tái', 'hiện', '.', '<eos>', '<pad>', '<pad>']


In [ ]:
# your code goes here

